In [16]:
import urllib
import time
import shutil
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [2]:
# from http://accent.gmu.edu/browse_language.php, return list of languages
def get_languages():
    """ This will return all the languages present in website
        http://accent.gmu.edu"""

    url = "http://accent.gmu.edu/browse_language.php"
    html = get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    languages = []
    language_lists = soup.findAll('ul', attrs={'class': 'languagelist'})
    for ul in language_lists:
        for li in ul.findAll('li'):
            languages.append(li.text)
    return languages

In [8]:
langs = get_languages()

In [9]:
langs

['aceh',
 'afrikaans',
 'agni',
 'agny',
 'akan',
 'albanian',
 'amazigh',
 'american sign language',
 'amharic',
 'ancient greek',
 'antigua and barbuda creole english',
 'anyin',
 'appolo',
 'arabic',
 'aramaic',
 'armenian',
 'aromanian',
 'ashanti',
 'asl',
 'azerbaijani',
 'azerbaijani, south',
 'azeri turk',
 'babur',
 'bafang',
 'baga',
 'bahasa indonesia',
 'bai',
 'balant',
 'balanta ganja',
 'bamanankan',
 'bambara',
 'bamun',
 'banganthe',
 'bangla',
 'baoule',
 'bari',
 'basque',
 'bassa',
 'bavarian',
 'belarusan',
 'bengali',
 'bikol',
 'bislama',
 'bosnian',
 'bulgarian',
 'burmese',
 'cameroon creole english',
 'cantonese',
 'carolinian',
 'castellano',
 'catalan',
 'cebuano',
 'chaam',
 'chagga',
 'chaldean',
 'chaldean neo aramaic',
 'chamorro',
 'charapa-spanish',
 'chichewa',
 'chin, mizo',
 'chinese',
 'chittagonian',
 'chuukese',
 'classical greek',
 'cotocoli',
 'creole',
 'creole french',
 'crioulo',
 'croatian',
 'czech',
 'danish',
 'dari',
 'darija',
 'dholuo

In [3]:
# from list of languages, return urls of each language landing page
def lang_pages(lst):
    """ This will return all the landing pages for the list of languages obtained
        from the above function"""

    urls=[]
    for lang in lst:
        urls.append('http://accent.gmu.edu/browse_language.php?function=find&language={}'.format(lang))
    return urls

In [10]:
langs_url = lang_pages(langs)

In [11]:
langs_url

['http://accent.gmu.edu/browse_language.php?function=find&language=aceh',
 'http://accent.gmu.edu/browse_language.php?function=find&language=afrikaans',
 'http://accent.gmu.edu/browse_language.php?function=find&language=agni',
 'http://accent.gmu.edu/browse_language.php?function=find&language=agny',
 'http://accent.gmu.edu/browse_language.php?function=find&language=akan',
 'http://accent.gmu.edu/browse_language.php?function=find&language=albanian',
 'http://accent.gmu.edu/browse_language.php?function=find&language=amazigh',
 'http://accent.gmu.edu/browse_language.php?function=find&language=american sign language',
 'http://accent.gmu.edu/browse_language.php?function=find&language=amharic',
 'http://accent.gmu.edu/browse_language.php?function=find&language=ancient greek',
 'http://accent.gmu.edu/browse_language.php?function=find&language=antigua and barbuda creole english',
 'http://accent.gmu.edu/browse_language.php?function=find&language=anyin',
 'http://accent.gmu.edu/browse_language

In [4]:
# from language, get the number of speakers of that language
def get_num(language):
    url = 'http://accent.gmu.edu/browse_language.php?function=find&language=' + language
    html = get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    test = soup.find_all('div', attrs={'class': 'content'})
    try:
        num = int(test[0].find('h5').text.split()[2])
    except AttributeError:
        num = 0
    return num

In [5]:
# from list of languages, return list of tuples (LANGUAGE, LANGUAGE_NUM_SPEAKERS) for mp3getter, ignoring languages
# with 0 speakers
def get_formatted_languages(languages):
    formatted_languages = []
    for language in languages:
        num = get_num(language)
        if num != 0:
            formatted_languages.append((language,num))
    return pd.DataFrame(formatted_languages, columns=["Language", "No. Of Speakers"]).sort_values(by="No. Of Speakers", ascending=False).reset_index(drop=True)

In [12]:
num_speakers = get_formatted_languages(langs)

In [14]:
num_speakers.head()

,Language,No. Of Speakers
0,english,626
1,spanish,216
2,chinese,193
3,arabic,170
4,mandarin,130


In [6]:
#For getting the speaker ids
def get_speaker_id(lang):
    speaker_id = []
    url = "http://accent.gmu.edu/browse_language.php?function=find&language=" + lang
    html = get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    for link in soup.find_all('a', href=re.compile("^browse_language")):
        href = link['href']
        url = urllib.parse.urlparse(href)
        params = urllib.parse.parse_qs(url.query)
        if 'speakerid' in params:
            speaker_id.append((params['speakerid'][0], "http://chnm.gmu.edu/accent/soundtracks/" + link.contents[0].replace(",", "") + ".mp3"))
    return pd.DataFrame(speaker_id, columns=["Speaker ID", "MP3 URL"]).reset_index(drop=True)

In [17]:
speaker_url = get_speaker_id('english')

In [18]:
speaker_url

,Speaker ID,MP3 URL
0,61,http://chnm.gmu.edu/accent/soundtracks/english...
1,77,http://chnm.gmu.edu/accent/soundtracks/english...
2,88,http://chnm.gmu.edu/accent/soundtracks/english...
3,99,http://chnm.gmu.edu/accent/soundtracks/english...
4,110,http://chnm.gmu.edu/accent/soundtracks/english...
5,121,http://chnm.gmu.edu/accent/soundtracks/english...
6,132,http://chnm.gmu.edu/accent/soundtracks/english...
7,143,http://chnm.gmu.edu/accent/soundtracks/english...
8,154,http://chnm.gmu.edu/accent/soundtracks/english...
9,62,http://chnm.gmu.edu/accent/soundtracks/english...


In [7]:
def get_mp3(url):
    urllib.request.urlretrieve(url, "./file_name.mp3")

In [24]:
x = speaker_url['MP3 URL'][1]

In [25]:
get_mp3(x)